In [ ]:
# Install GDAL and Geopandas
!apt install gdal-bin python-gdal python3-gdal --quiet
!apt install python3-rtree --quiet
!pip install git+git://github.com/geopandas/geopandas.git --quiet
!pip install descartes --quiet
!pip install folium

In [ ]:
import pandas as pd
import geopandas as gpd 
import matplotlib.pyplot as plt
import folium 
from shapely.ops import nearest_points
from shapely.geometry import LineString

In [ ]:
covid=pd.read_csv("time_series_covid19_deaths_global.csv")
covid.head()

In [ ]:
air=pd.read_csv("../2020_combined.csv")
air.head()

In [ ]:
def create_gdf(df, x="Lat", y="Long"):
    return gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df[x], df[y]), crs={"init":"epsg:4326"})